Import dos módulos necessários e declaração de constantes
---

In [1]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

# quantidade de canais que as imagens de entrada possuem
# escalar cor de cinza, entao 1
CHANNEL = 1
# valor da largura e da altura das imagens de entrada
WIDTH_HEIGHT = 20

Using TensorFlow backend.


Load do arquivo de dados
---

In [2]:
load = np.loadtxt('exdata.csv', delimiter=',')

# cada coluna tem um padrao de digito
data = load[:-1].T

# a ultima linha eh a classificacao do digito
result = load[-1] 
# digito 0 corresponde ao valor 10
result[result == 10] = 0

Pré-processamento dos dados e das classes
---

In [3]:
# trasformar cada linha (digito) em uma matriz 20 x 20 x 1
data = data.reshape(data.shape[0], WIDTH_HEIGHT, WIDTH_HEIGHT, CHANNEL)

# converte array de 1 dimensao para uma matriz de dimensao 10
# ou seja, criar 10 classes, uma para cada digito possivel
result = keras.utils.to_categorical(result, 10)

Separação dos dados em treinamento e teste
---

In [4]:
in_train, in_test, out_train, out_test = train_test_split(data, 
                                                          result,
                                                          test_size=(25/100),
                                                          train_size=(75/100))

Definição da arquitetura da rede neural
---

In [5]:
# 'pilha' de camadas lineares
model = Sequential()

# primeira camada precisa saber o que espera de entrada
# Conv2D cria uma camada de 'convolution' (add cada elemento da imagem com o seu vizinho local)
# isso eh feito atraves do input_shape
# 32 eh o numero de filtros
# relu = rectified linear unit
model.add(Conv2D(32, 
                 kernel_size=(3, 3), 
                 activation='relu', 
                 input_shape=(WIDTH_HEIGHT, WIDTH_HEIGHT, CHANNEL)))
model.add(Conv2D(64, (3, 3), activation='relu'))

# MaxPooling2D cria uma camada que faz um processo de discretizacao baseada em amostra
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout cria uma camada de regularizacao
# 0.25 eh a fracao da quantidade de entrada que entrara na camada
model.add(Dropout(0.25))


model.add(Flatten())

# Dense cria uma camada que representa uma multiplicacao de matrizes
# 128 eh a dimensionalidade da saida
model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

Compilação da rede neural
---

In [6]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Input dos dados de treinamento
---

In [7]:
model.fit(in_train, out_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(in_test, out_test))

Train on 3750 samples, validate on 1250 samples
Epoch 1/10
3750/3750 [==============================] - 2s 626us/step - loss: 1.4240 - acc: 0.5333 - val_loss: 0.4524 - val_acc: 0.8840
Epoch 2/10
3750/3750 [==============================] - 2s 636us/step - loss: 0.4785 - acc: 0.8592 - val_loss: 0.2662 - val_acc: 0.9224
Epoch 3/10
3750/3750 [==============================] - 2s 588us/step - loss: 0.3105 - acc: 0.9080 - val_loss: 0.1726 - val_acc: 0.9504
Epoch 4/10
3750/3750 [==============================] - 2s 580us/step - loss: 0.2254 - acc: 0.9355 - val_loss: 0.1612 - val_acc: 0.9536
Epoch 5/10
3750/3750 [==============================] - 2s 584us/step - loss: 0.1815 - acc: 0.9504 - val_loss: 0.1715 - val_acc: 0.9456
Epoch 6/10
3750/3750 [==============================] - 2s 585us/step - loss: 0.1585 - acc: 0.9520 - val_loss: 0.1212 - val_acc: 0.9664
Epoch 7/10
3750/3750 [==============================] - 2s 625us/step - loss: 0.1370 - acc: 0.9568 - val_loss: 0.1187 - val_acc: 0.9608


Avaliação com os dados de teste
---

In [8]:
score = model.evaluate(in_test, out_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.09487587989544263
Test accuracy: 0.9688
